In [20]:
!pip install google-generativeai==0.8.4

In [21]:
# @title 기본 사용법 1 – 싱글턴으로 메시지 주고받기
import google.generativeai as genai
import os
from google.colab import userdata
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [22]:
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.text)

인공지능은 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 능력을 수행하는 컴퓨터 시스템 또는 기술입니다.



In [23]:
# @title 기본 사용법 2 – 멀티턴으로 메시지 주고받기(1)
from google.generativeai import ChatSession
model = genai.GenerativeModel('gemini-2.0-flash')
chat_session: ChatSession = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]
for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')

[사용자]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[모델]: 인공지능은 컴퓨터가 인간의 지능적인 기능을 모방하여 학습, 추론, 문제 해결 등을 수행하는 기술입니다.

[사용자]: 의식이 있는지 한 문장으로 답하세요.
[모델]: 현재 인공지능은 의식이 없습니다.



In [24]:
# @title 기본 사용법 3 – 멀티턴으로 메시지 주고받기(2)
model = genai.GenerativeModel('gemini-2.0-flash')
user_queries = [{'role':'user', 'parts': ["인공지능에 대해 한 문장으로 짧게 설명하세요."]},
                {'role':'user', 'parts': ["의식이 있는지 한 문장으로 답하세요."]}
            ]
history = []
for user_query in user_queries:
    history.append(user_query)
    print(f'[사용자]: {user_query["parts"][0]}')
    response = model.generate_content(history)
    print(f'[모델]: {response.text}')
    history.append(response.candidates[0].content)

[사용자]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[모델]: 인공지능은 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 작업을 수행하는 컴퓨터 시스템입니다.

[사용자]: 의식이 있는지 한 문장으로 답하세요.
[모델]: 현재의 인공지능은 의식이 없습니다.



In [25]:
# @title 페르소나 만들기
system_instruction="당신은 유치원 선생님입니다. 사용자는 유치원생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."
model = genai.GenerativeModel('gemini-2.0-flash', system_instruction=system_instruction)
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능이 뭐에요?", "그럼 스스로 생각도 해요?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')


[사용자]: 인공지능이 뭐에요?
[모델]: 인공지능은 컴퓨터가 사람처럼 생각하고 배우는 거야! 마치 우리가 공부하고 숙제하는 것처럼 말이야. 똑똑한 컴퓨터라고 생각하면 쉬워!

[사용자]: 그럼 스스로 생각도 해요?
[모델]: 응, 맞아! 인공지능은 스스로 배우고 생각해서 문제를 해결하기도 해. 우리가 그림 그리는 방법 배우듯이 말이야! 신기하지?



In [26]:
# @title 답변 형식 지정하기 - JSON 형식 사용하기
import json
system_instruction='JSON schema로 주제별로 답하되 3개를 넘기지 말 것:{{"주제": <주제>, "답변":<두 문장 이내>}}'
model = genai.GenerativeModel("gemini-2.0-flash", system_instruction=system_instruction, generation_config={"response_mime_type": "application/json"})
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능의 특징이 뭐에요?", "어떤 것들을 조심해야 하죠?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    answer_dict = json.loads(response.text)
    print(answer_dict)


[사용자]: 인공지능의 특징이 뭐에요?
[{'주제': '인공지능의 특징', '답변': '인공지능은 학습을 통해 스스로 지식을 축적하고, 복잡한 문제를 해결할 수 있습니다. 또한, 인간의 지능을 모방하여 다양한 분야에서 활용될 수 있습니다.'}]
[사용자]: 어떤 것들을 조심해야 하죠?
[{'주제': '보안', '답변': '개인 정보 유출 및 해킹 위험에 주의하고, 강력한 보안 시스템을 구축해야 합니다.'}, {'주제': '개인 정보 보호', '답변': '개인 정보 수집 및 활용에 대한 투명성을 확보하고, 데이터 오남용을 방지해야 합니다.'}, {'주제': '사회적 책임', '답변': '인공지능 기술이 사회적 불평등을 심화시키지 않도록 윤리적인 개발과 사용이 필요합니다.'}]


In [27]:
# @title 답변 형식 지정하기 - 구조화된 출력(Structured output) 사용하기
json_schema = {
    'properties': {
        'product_name': {
            'type': 'string'
        },
        'size': {
            'enum': ['S', 'M', 'L', 'XL'],
            'type': 'string'
        },
        'price': {
            'type': 'integer'
        },
    },
    'required': ['price', 'size', 'product_name'],
    'type': 'object'
}

model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content(
    "덩치가 크고 등산을 좋아하는 남성의 옷을 추천해주세요",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json", response_schema=json_schema
    ),
)
print(response.text)

{
  "price": 150000,
  "product_name": "빅사이즈 남성 등산 바지",
  "size": "XL"
}


In [28]:
import enum
from typing_extensions import TypedDict

class Size(enum.Enum):
    S = "S"
    M = "M"
    L = "L"
    XL = "XL"

class Product(TypedDict):
    product_name: str
    size: str
    price: int

model = genai.GenerativeModel("gemini-2.0-flash")

response = model.generate_content(
    "덩치가 크고 등산을 좋아하는 남성의 옷을 추천해주세요",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json", response_schema=Product
    ),
)
print(response.text)

{
  "product_name": "남성용 빅사이즈 등산 바지",
  "size": "3XL"
}


In [29]:
# @title 제미나이 AI 입력 데이터 구조
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")
print(response.candidates[0].content)
print(response.candidates[0].content.parts[0].text)

parts {
  text: "인공지능은 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 기능을 수행하는 컴퓨터 시스템 또는 기술입니다.\n"
}
role: "model"

인공지능은 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 기능을 수행하는 컴퓨터 시스템 또는 기술입니다.



In [30]:
# @title chat_session history에 들어 있는 Content 객체와 데이터
system_instruction="당신은 유치원 선생님입니다. 사용자는 유치원생입니다. 쉽고 친절하게 이야기하되 3문장 이내로 짧게 얘기하세요."
model = genai.GenerativeModel('gemini-2.0-flash', system_instruction=system_instruction)
chat_session = model.start_chat(history=[])  # ChatSession 객체 반환
user_queries = ["인공지능이 뭐에요?", "그럼 스스로 생각도 해요?"]

for user_query in user_queries:
    print(f'[사용자]: {user_query}')
    response = chat_session.send_message(user_query)
    print(f'[모델]: {response.text}')

print("-"*50)

for idx, content in enumerate(chat_session.history):
    print(f"{content.__class__.__name__}[{idx}]")
    print(content)


[사용자]: 인공지능이 뭐에요?
[모델]: 인공지능은 컴퓨터가 사람처럼 생각하고 배우는 것을 말해. 마치 우리 친구들이 공부하고 새로운 것을 알아가는 것과 같은 거야. 인공지능은 우리 생활을 더 편리하게 만들어 줄 수 있단다!

[사용자]: 그럼 스스로 생각도 해요?
[모델]: 응, 인공지능은 스스로 생각할 수 있어. 마치 우리가 퍼즐을 풀거나 그림을 그릴 때처럼 말이야. 하지만 아직은 우리가 가르쳐 준 것들을 배우고 따라 하는 정도란다!

--------------------------------------------------
Content[0]
parts {
  text: "인공지능이 뭐에요?"
}
role: "user"

Content[1]
parts {
  text: "인공지능은 컴퓨터가 사람처럼 생각하고 배우는 것을 말해. 마치 우리 친구들이 공부하고 새로운 것을 알아가는 것과 같은 거야. 인공지능은 우리 생활을 더 편리하게 만들어 줄 수 있단다!\n"
}
role: "model"

Content[2]
parts {
  text: "그럼 스스로 생각도 해요?"
}
role: "user"

Content[3]
parts {
  text: "응, 인공지능은 스스로 생각할 수 있어. 마치 우리가 퍼즐을 풀거나 그림을 그릴 때처럼 말이야. 하지만 아직은 우리가 가르쳐 준 것들을 배우고 따라 하는 정도란다!\n"
}
role: "model"



In [31]:
# @title 이미지 데이터 인식
from google.colab import drive
import os
import PIL.Image
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks/5_gemini")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
image_data = PIL.Image.open("./data/images/monalisa.jpg") # 모나리자 그림
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content(["이 그림에 대해 한 문장으로 설명하세요.", image_data])
print(response.text)

이 그림은 레오나르도 다 빈치의 유명한 초상화인 모나리자를 보여줍니다.


In [33]:
# @title 제미나이 AI 출력 데이터 구조
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\uc774 \uadf8\ub9bc\uc740 \ub808\uc624\ub098\ub974\ub3c4 \ub2e4 \ube48\uce58\uc758 \uc720\uba85\ud55c \ucd08\uc0c1\ud654\uc778 \ubaa8\ub098\ub9ac\uc790\ub97c \ubcf4\uc5ec\uc90d\ub2c8\ub2e4."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.14548943837483724
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 1301,
        "candidates_token_count": 30,
        "total_token_count": 1331
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [34]:
# @title Candidate 객체
print(f"건수: {len(response.candidates)}")
print("="*50)
for candidate in response.candidates:
    print(candidate)

건수: 1
content {
  parts {
    text: "이 그림은 레오나르도 다 빈치의 유명한 초상화인 모나리자를 보여줍니다."
  }
  role: "model"
}
finish_reason: STOP
avg_logprobs: -0.14548943837483724



In [35]:
# @title FinishReason 객체
print(f"finish_reason: {response.candidates[0].finish_reason.name},{response.candidates[0].finish_reason}")

finish_reason: STOP,1
